<a href="https://colab.research.google.com/github/jdasam/mas1004-2022/blob/main/notebooks/Data_AI_9th_week_image.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# !pip install -q jmd_imagescraper
# from jmd_imagescraper.core import duckduckgo_search

from pathlib import Path
import torch
import torchvision
import torchvision.transforms as transforms
from PIL import Image
from tqdm.auto import tqdm

/home/teo/.local/lib/python3.8/site-packages/pkg_resources/__init__.py:123: PkgResourcesDeprecationWarning: 0.23ubuntu1 is an invalid version and will not be supported in a future release
  warnings.warn(


In [ ]:
!wget https://artbench.eecs.berkeley.edu/files/artbench-10-imagefolder-split.tar
!tar -xvf artbench-10-imagefolder-split.tar

In [ ]:
# How to load image

img_path = Path('artbench-10-imagefolder-split/test/expressionism/rudolf-lang_hillside-1953.jpg')
img = Image.open(img_path).convert('RGB')

In [ ]:
pil2tensor = transforms.Compose([
                        transforms.Resize(256),
                        transforms.CenterCrop(224),
                        transforms.ToTensor(),
                        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])

tensor2pil = transforms.Compose([
    transforms.Normalize(mean=[0, 0, 0], std=[4.3668, 4.4643, 4.4444]),
    transforms.Normalize(mean=[-0.485, -0.456, -0.406], std=[1, 1, 1]),
    transforms.ToPILImage()
])


In [2]:
model = torchvision.models.resnet18(pretrained=True)
